In [10]:
import json
import pandas as pd 

In [8]:
def convert_list_to_dict(obj):
    if isinstance(obj, dict):
        for key, value in obj.items():
            obj[key] = convert_list_to_dict(value)
    elif isinstance(obj, list) and all(isinstance(item, dict) for item in obj):
        combined_dict = {}
        for item in obj:
            combined_dict.update(item)
        return combined_dict
    return obj

In [22]:
def get_camera_configuration(data) :
    try : 
        if 'Single' in list(data.keys()) :
            return 'Single'
        elif 'Dual' in list(data.keys()) :
            return 'Dual'
        elif 'Triple' in list(data.keys()) :
            return 'Triple'
        elif 'Quad' in list(data.keys()) :
            return 'Quad'
        elif 'Penta' in list(data.keys()) :
            return 'Penta'
        elif 'Hexa' in list(data.keys()) :
            return 'Hexa'
        else : 
            return None 
    except :
        return None 

def Loudspeaker(data) :
    try : 
        if data['Loudspeaker'].split(',')[0] == 'Yes' :
            return True 
        return False 
    except : 
        return False

def Has_jack(data) :
    try : 
        if data['3.5mm jack'].split(',')[0] == 'Yes' :
            return True 
        return False
    except :
        return False

def get_WLAN(data) :
    try : 
        return data['WLAN'].split(',')[0]
    except :
        return None

def get_bluetooth_version(data) :
    try : 
        if data['Bluetooth'].split(',')[0] != 'No' :
            return data['Bluetooth'].split(',')[0].split(' ')[0]
        else :
            return None
    except :
        return None 

def Has_NFC(data) :
    try : 
        if data['NFC'][:3] == 'Yes' :
            return True 
        return False
    except :
        return False 


In [23]:
def fill_df(json_file) :
    
    file = open(json_file)
    data = json.load(file)

    brand = list(data.keys())[0]

    device_list = list(data[brand].keys())

    df = pd.DataFrame(index = device_list , columns = ['main_camera_configuration' , 'selfie_camera_configuration' , 'has_loudspeaker' , 
                                                       'has_jack','WLAN' , 'NFC' , 'bluetooth_version'])
    
    for device in device_list[:] : 
        try :
            d = data[brand][device]['Main Camera']
            d = convert_list_to_dict(d)
            #print(d)
            df.loc[device]['main_camera_configuration'] = get_camera_configuration(d)
        except : 
            df.loc[device]['main_camera_configuration'] = None

        try : 
            d  = data[brand][device]['Selfie camera']
            d = convert_list_to_dict(d)
            #print(d)
            df.loc[device]['selfie_camera_configuration'] = get_camera_configuration(d)
        except : 
            df.loc[device]['selfie_camera_configuration'] = None

        try : 
            d = data[brand][device]['Sound']
            d = convert_list_to_dict(d)
            #print(d)
            df.loc[device]['has_jack'] = Has_jack(d)
            df.loc[device]['has_loudspeaker'] = Loudspeaker(d)
        except : 
            df.loc[device]['has_jack'] = None
            df.loc[device]['has_loudspeaker'] = None

        try : 
            d = data[brand][device]['Comms']
            d = convert_list_to_dict(d)
            #print(d)
            df.loc[device]['WLAN'] = get_WLAN(d)
            df.loc[device]['bluetooth_version'] = get_bluetooth_version(d)
            df.loc[device]['NFC'] = Has_NFC(d)
        except :
            df.loc[device]['WLAN'] = None
            df.loc[device]['bluetooth_version'] = None
            df.loc[device]['NFC'] = None
        
    return df 
    


In [88]:
def make_sensor_df(json_file) :
    
    file = open(json_file)
    data = json.load(file)

    brand = list(data.keys())[0]

    device_list = list(data[brand].keys()) 

    for device in device_list[:] : 
        sensor_list = [ sensor.strip() for sensor in data[brand][device]['Features'][0]['Sensors'].split(',') ]
        #print(sensor_list)
        for sensor in sensor_list :
            sensor_df.loc[len(sensor_df)] = {'device': device, 'sensor': sensor}

In [89]:
def make_USB_df(json_file) :
    
    file = open(json_file)
    data = json.load(file)

    brand = list(data.keys())[0]

    device_list = list(data[brand].keys()) 

    #convert_list_to_dict(data[brand][device]['Comms'])['USB'].split(',')
    for device in device_list[:] : 
        USB_list = [ USB.strip()  for USB in convert_list_to_dict(data[brand][device]['Comms'])['USB'].split(',') if USB.strip()!= 'No'  ]
        #print(USB_list)
        for USB in USB_list :
            USB_df.loc[len(USB_df)] = {'device' : device  , 'USB' : USB }

In [90]:
files = ['Alcatel_data.json','Apple_data.json' , 'Asus_data.json' , 'Blu_data.json' , 'HTC_data.json' , 'Huawei_data.json' , 'Infinix_data.json' , 'Lenovo_data.json' ,
         'LG_data.json' , 'Nokia_data.json' , 'Samsung_data.json' , 'Sony_data.json' , 'Xiaomi_data.json' , 'ZTE_data.json']

In [102]:
df = pd.DataFrame(columns = ['main_camera_configuration' , 'selfie_camera_configuration' , 'has_loudspeaker' , 
                                                       'has_jack','WLAN' , 'NFC' , 'bluetooth_version'])
for file in files[:] :
    new_df = fill_df(file)
    df = pd.concat([df , new_df] , axis=0)

In [103]:
df

,main_camera_configuration,selfie_camera_configuration,has_loudspeaker,has_jack,WLAN,NFC,bluetooth_version
1B (2022),Single,Single,True,True,Wi-Fi 802.11 b/g/n,False,5.0
1L Pro (2021),Dual,Single,True,True,Wi-Fi 802.11 b/g/n,False,4.2
1 (2021),Single,Single,True,True,Wi-Fi 802.11 b/g/n,False,4.2
3L (2021),Triple,Single,True,True,Wi-Fi 802.11 b/g/n,True,5.0
1S (2021),Triple,Single,True,True,Wi-Fi 802.11 b/g/n,False,5.0
...,...,...,...,...,...,...,...
F101,Single,Single,True,False,No,False,2.1
F100,Single,Single,True,False,No,False,2.1
Coral200 Sollar,None,None,False,False,No,False,None
Blade V20,Triple,Single,True,True,Wi-Fi 802.11 b/g/n/ac,False,4.2


In [110]:
df.rename_axis('Device' , inplace= True)

In [111]:
df.to_csv('Device_df_part1.csv')

In [112]:
pd.read_csv('Device_df_part1.csv')

,Device,main_camera_configuration,selfie_camera_configuration,has_loudspeaker,has_jack,WLAN,NFC,bluetooth_version
0,1B (2022),Single,Single,True,True,Wi-Fi 802.11 b/g/n,False,5.0
1,1L Pro (2021),Dual,Single,True,True,Wi-Fi 802.11 b/g/n,False,4.2
2,1 (2021),Single,Single,True,True,Wi-Fi 802.11 b/g/n,False,4.2
3,3L (2021),Triple,Single,True,True,Wi-Fi 802.11 b/g/n,True,5.0
4,1S (2021),Triple,Single,True,True,Wi-Fi 802.11 b/g/n,False,5.0
...,...,...,...,...,...,...,...,...
5717,F101,Single,Single,True,False,No,False,2.1
5718,F100,Single,Single,True,False,No,False,2.1
5719,Coral200 Sollar,NaN,NaN,False,False,No,False,NaN
5720,Blade V20,Triple,Single,True,True,Wi-Fi 802.11 b/g/n/ac,False,4.2


In [104]:
sensor_df = pd.DataFrame(columns = ['device' , 'sensor'])
for file in files : 
    make_sensor_df(file)

In [115]:
sensor_df.set_index('device' , inplace=True)

In [116]:
sensor_df.to_csv('Sensor_df.csv')

In [106]:
USB_df = pd.DataFrame(columns = ['device' , 'USB'])
for file in files :
    make_USB_df(file)

In [118]:
USB_df.set_index('device' , inplace=True)

In [120]:
USB_df.to_csv('USB_df.csv')